In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from utils import *
import re
import datetime as dt

In [2]:
train = pd.read_csv('data/train_master.csv').fillna(0)
test = pd.read_csv('data/test_master.csv').fillna(0)
sample = pd.read_csv('data/sample_submission.csv')

In [3]:
params = {}
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['eval_metric'] = 'mlogloss'
params['eta'] = 0.05
params['max_depth'] = 8
params['colsample_bytree'] = 0.3
params['silent'] = 1
num_round = 500

In [5]:
xgmat = xgb.DMatrix(train.iloc[:, 1:], label=train.loc[:, 'fault_severity'])
bst = xgb.train(params, xgmat, num_round)

In [8]:
testmat = xgb.DMatrix(test.iloc[:, 2:])

In [9]:
prob = bst.predict(testmat)

In [10]:
prob

array([[  9.87458169e-01,   1.21495444e-02,   3.92277172e-04],
       [  1.99917555e-01,   5.51306084e-02,   7.44951844e-01],
       [  9.94599462e-01,   5.36156865e-03,   3.89436100e-05],
       ..., 
       [  1.53612763e-01,   8.38197470e-01,   8.18978157e-03],
       [  9.68703687e-01,   3.12639661e-02,   3.23433014e-05],
       [  9.71620262e-01,   2.80892085e-02,   2.90460273e-04]], dtype=float32)

In [11]:
xgbc_prob = pd.DataFrame({
    'id': sample.loc[:, 'id'].values, 
    'predict_0': prob[:, 0],
    'predict_1': prob[:, 1],
    'predict_2': prob[:, 2]})

In [12]:
xgbc_prob.to_csv('xgbc.csv', index=False)